# Aprendizaje no supervisado

## 1. Importar bibliotecas necesarias

In [ ]:
import pandas as pd
import numpy as np

## 2. Cargar el conjunto de datos

In [ ]:
# Establece tu ruta de trabajo
import os
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ruta = os.path.join('..', 'docs', 'datos', 'wine.csv')

In [ ]:
#ruta = os.path.join('drive', 'MyDrive', 'curso-machine-learning', 'wine.csv') #content

In [ ]:
df = pd.read_csv(ruta)
df

> El conjunto de datos contiene 3 variables cuantitativas. Sobre características de vinos:
> - ``Alcohol``
> - ``Proline``
> - ``flavanoids``
>

> **No contiene la variable objetivo**, ya que en el aprendizaje no supervisado no se cuenta con dicha variable.

> Supongamos que estamos interesados en agrupar los datos en función de estas características. Para ello, podemos utilizar técnicas de **clustering como K-means**.

## K-means Clustering  

Es un **algoritmo de aprendizaje no supervisado** que agrupa observaciones en **_k_ clusters** basándose en su similitud. 
 
Cada cluster se define por un **centroide** (el “promedio” de los puntos asignados a ese grupo).  


### Idea principal
- Queremos que los puntos dentro de un mismo cluster estén lo más **cercanos** posible al centroide.  
- Y que los clusters estén lo más **separados** posible entre sí.  

### Algoritmo paso a paso
1. **Elegir k** (número de clusters).  
2. **Inicializar centroides** (de forma aleatoria).  
3. **Asignar puntos a clusters**:  
   Cada punto se asigna al centroide más cercano usando distancia euclidiana:  

   $$
   d(x, \mu_j) = \sqrt{\sum_i (x_i - \mu_{j,i})^2}
   $$

4. **Actualizar centroides**:  

* Una vez asignados los puntos, el nuevo centroide de cada cluster se obtiene como el promedio de todos los puntos en ese cluster.

* Es decir, el centroide “se mueve” hacia el centro real de los puntos que lo rodean.

5. **Iterar** pasos 3 y 4 hasta que los centroides ya no cambien (o hasta un número máximo de iteraciones).

![kmeans-convergence](../docs/_static/512px-K-means_convergence.gif)

**Figura 1:** Convergencia del algoritmo K-means. Fuente: [Wikipedia](https://commons.wikimedia.org/wiki/File:K-means_convergence.gif). Chire, CC BY-SA 4.0 <https://creativecommons.org/licenses/by-sa/4.0>, via Wikimedia Commons.

### Función objetivo
K-means minimiza la **suma de distancias cuadradas** entre puntos y sus centroides:

$$
J = \sum_{j=1}^k \sum_{x \in C_j} \| x - \mu_j \|^2
$$

- $C_j$ = conjunto de puntos del cluster $j$  
- $\mu_j$ = centroide del cluster $j$

K-Means suma todas esas distancias (al cuadrado) y trata de que el total sea lo más pequeño posible.

## 3. Características de los datos

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# --- Histogramas ---#
for col in df.columns:
    fig, ax = plt.subplots(figsize=(6,3))
    sns.histplot(df[col],
                 kde=True, 
                 bins=20, 
                 ax=ax, 
                 color='purple', 
                 alpha=0.4)
    ax.set_title(f"Distribución de {col}")
    plt.show()

In [ ]:
# --- Boxplots ---#
plt.figure(figsize=(8,4))
sns.boxplot(data=df)
plt.title("Boxplots de variables")
plt.show()

In [ ]:
# --- Pairplot ---#
sns.pairplot(df)
plt.show()

In [ ]:
# --- Matriz de correlación ---#
plt.figure(figsize=(6,5))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Matriz de correlación")
plt.show()

Recordemos que la matriz de correlación nos ayuda a observar dos cosas importantes:

* La **magnitud** de la correlación entre variables (qué tan fuerte es la relación).
* La **dirección** de la correlación (si es positiva o negativa).

También es una buena herramienta para detectar **multicolinealidad** (cuando dos o más variables están altamente correlacionadas entre sí).

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# --- Scatter 3D (aun sin clusterizar) ---#
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['flavanoids'], df['alcohol'], df['proline'])

ax.set_xlabel('Flavanoids')
ax.set_ylabel('Alcohol')
ax.set_zlabel('Proline')

plt.title("Scatter 3D de las variables originales")
plt.show()

## Preprocesamiento de datos

In [ ]:
# --- Escalamiento de datos --- #
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(X_scaled, columns=df.columns, index=df.index)

In [ ]:
#StandardScaler?

In [ ]:
df_scaled.head()

In [ ]:
# --- Boxplots de datos escalados ---#
plt.figure(figsize=(8,4))
sns.boxplot(data=df_scaled)
plt.title("Boxplots de variables (escaladas)")
plt.show()

## Previas para conocer k

In [ ]:
# Antes de aplicar KMeans, debemos elegir el número de clusters (k)
from sklearn.cluster import KMeans

sum_distancias = []
ks = range(1, 11) # Probamos k desde 1 hasta 10

for k in ks:
    modelo = KMeans(n_clusters=k, random_state=42, n_init=11)
    modelo.fit(X_scaled)
    sum_distancias.append(modelo.inertia_) # Guardamos la suma de distancias de cada punto a su centroide

In [ ]:
# --- Graficar el codo ---#
plt.plot(ks, sum_distancias, marker='o')
plt.xlabel("Número de clusters (k)")
plt.ylabel("Suma de distancias a los centroides")
plt.title("Método del codo")
plt.show()

Además del _método del codo_, hay una métrica que nos ayuda a complementar nuestro análisis para elegir el número óptimo de clusters: el **_silhouette score_**.

## 📊 Silhouette Analysis

El **silhouette analysis** se puede usar para estudiar la distancia de separación entre los clusters resultantes.  

El **silhouette plot** muestra qué tan cerca está cada punto de un cluster respecto a los puntos de los clusters vecinos.  

De esta manera, ofrece una forma visual de evaluar parámetros como el **número de clusters**.


### Intuición

El coeficiente de silhouette para un punto $i$ se define como:

$$
s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}}
$$

donde:
- $a(i)$ = distancia promedio del punto $i$ a todos los puntos de su propio cluster (**cohesión**).  
- $b(i)$ = distancia promedio del punto $i$ al cluster vecino más cercano (**separación**).  

El valor resultante está en el rango $[-1, 1]$.

### Interpretación

- **Cerca de +1** → el punto está bien asignado, lejos de los clusters vecinos.  
- **≈ 0** → el punto se encuentra en la frontera entre dos clusters.  
- **Negativo (< 0)** → el punto podría estar mal asignado (más cerca de otro cluster que del suyo).

Puedes revisar la documentación oficial de Silhouette Score en [sklearn](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html).

In [ ]:
from sklearn.metrics import silhouette_score

# Evalúa k candidatos del codo
for k in [2, 3, 4]:
    km = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = km.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)
    print(f"k={k}, silhouette={sil:.3f}")

In [ ]:
from sklearn.metrics import silhouette_samples

def plot_silhouette(X, k=3, ax=None):
    # Ajustar modelo
    km = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = km.fit_predict(X)

    # Calcular coeficientes de silhouette
    sil_avg = silhouette_score(X, labels)
    sil_values = silhouette_samples(X, labels)

    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 5))

    y_lower = 10
    for i in range(k):
        cluster_sil = sil_values[labels == i]
        cluster_sil.sort()
        size_cluster = cluster_sil.shape[0]
        y_upper = y_lower + size_cluster

        ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_sil, alpha=0.7)
        ax.text(-0.05, y_lower + 0.5 * size_cluster, str(i))
        y_lower = y_upper + 10

    ax.axvline(x=sil_avg, color="red", linestyle="--")
    ax.set_title(f"Silhouette plot (k={k}, avg={sil_avg:.3f})")
    ax.set_xlabel("Coeficiente silhouette")
    ax.set_ylabel("Cluster")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_silhouette(X_scaled, k=2, ax=axes[0])
plot_silhouette(X_scaled, k=3, ax=axes[1])
plt.show()

## Modelo K-means

In [ ]:
from sklearn.cluster import KMeans

# KMeans con k=3
k = 3
mi_modelo = KMeans(n_clusters=k,
                random_state=42, 
                n_init=10)

# Generamos nueva col para identificar el clúster
df["cluster"] = mi_modelo.fit_predict(X_scaled)                

labels = mi_modelo.fit_predict(X_scaled)

In [ ]:
# Centroides en el espacio escalado
centroides = mi_modelo.cluster_centers_
centroides

## Visualización de clústers

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df["flavanoids"], df["alcohol"], df["proline"],
           c=df["cluster"], cmap="viridis", s=50, alpha=0.7)

# centroides en las variables originales
centroides_orig = scaler.inverse_transform(centroides)
ax.scatter(centroides_orig[:, df.columns.get_loc("flavanoids")],
           centroides_orig[:, df.columns.get_loc("alcohol")],
           centroides_orig[:, df.columns.get_loc("proline")],
           c="red", marker="X", s=200, label="Centroides")

ax.set_xlabel("Flavanoids")
ax.set_ylabel("Alcohol")
ax.set_zlabel("Proline")
plt.title("Clústers K-means con k=3")
ax.legend()
plt.show()

In [ ]:
# Gráfico 3D interactivo
import plotly.express as px
import plotly.graph_objects as go

df_plot = df.copy()
df_plot['cluster'] = labels.astype(str)  

fig = px.scatter_3d(
    df_plot,
    x='flavanoids', y='alcohol', z='proline',
    color='cluster',
    size=[3]*len(df_plot),        
    title='Wine + K-means (3D)'
)

centroids_orig = scaler.inverse_transform(mi_modelo.cluster_centers_)
fig.add_trace(go.Scatter3d(
    x=centroids_orig[:, 0],
    y=centroids_orig[:, 1],
    z=centroids_orig[:, 2],
    mode='markers+text',
    marker=dict(size=5, symbol='x'),     
    text=[f'C{i}' for i in range(centroids_orig.shape[0])],
    textposition='top center',
    name='Centroides'
))

fig.show()

## Nuevos datos con modelo entrenado

In [ ]:
# Supongamos que tenemos nuevos vinos (valores sin escalar)
nuevos = [[2.0, 13.5, 750],   # (flavanoids, alcohol, proline)
          [1.0, 12.0, 400]]

In [ ]:
# Primero escalamos con el mismo scaler
scaler = StandardScaler()
nuevos_scaled = scaler.fit_transform(nuevos)

In [ ]:
# Predecimos su clúuster (nota que usamos el modelo ya entrenado [llamado: mi_modelo])
pred_nuevos = mi_modelo.predict(nuevos_scaled)
pred_nuevos

In [ ]:
# --- Graficar ---#
plt.figure(figsize=(7,6))
plt.scatter(X_scaled[:,0], X_scaled[:,1], c=labels, cmap="viridis", alpha=0.6, s=40, label="Datos originales")

# centroides
plt.scatter(mi_modelo.cluster_centers_[:,0], mi_modelo.cluster_centers_[:,1], 
            c="red", marker="X", s=200, label="Centroides")

# nuevos puntos
plt.scatter(nuevos_scaled[:,0], nuevos_scaled[:,1],
            c=pred_nuevos, cmap="viridis", edgecolors="k", s=200, marker="o", label="Nuevos puntos")

plt.xlabel("Flavanoids (escalado)")
plt.ylabel("Alcohol (escalado)")
plt.title("K-means con nuevos puntos asignados")
plt.legend()
plt.show()